In [1]:
pip install tensorflow


  Using cached tensorflow-2.16.1-cp310-cp310-win_amd64.whl.metadata (3.5 kB)
  Using cached tensorflow_intel-2.16.1-cp310-cp310-win_amd64.whl.metadata (5.0 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached gast-0.5.4-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached h5py-3.11.0-cp310-cp310-win_amd64.whl.metadata (2.5 kB)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl.metadata (5.3 kB)
  Using cached ml_dtypes-0.3.2-cp310-cp310-win_amd64.whl.metadata (20 kB)
  Using cached termcolor-2.4.0-py3-none-any.whl.metadata (6.1 kB)
  Using cached grpcio-1.64.1-cp310-cp310-win_amd64.whl.metadata (3.4 kB)
  Using cached tensorboard-2.16.2-py3-none-any.whl.metadata (1.6 kB)
  Using cached keras-3.3.3-py3-none-any.whl.metadata (5.7 kB)
  Using cached tensorflow_io_gcs_filesystem-0.31.0-cp310-cp310-win_amd64.whl.metadata (14 kB)
  Using cached rich-13.7.1-py3-none-any.whl.metada

In [1]:
import pandas as pd
import random
import numpy as np
import os
from sklearn.preprocessing import LabelEncoder
#0=incorrect 1 correct 0-wallsit 1-plank
#loading the dataset into array format 
data=pd.read_csv("squat.csv",header=1)
# Shuffle the rows of the DataFrame
shuffled = data.sample(frac=1).reset_index(drop=True)
#print(shuffled.head())
shuffled.shape

(466, 18)

In [2]:
from sklearn.model_selection import train_test_split
# Assuming the last column is the target variable (y) and the rest are features (X)
X = data.iloc[:, :-1]  # All columns except the last one
y = data.iloc[:, -1]   # Last column

# Split the dataset into training and testing sets (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape)


(372, 17)


In [3]:
#creating the neural network model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Input
from tensorflow.keras.optimizers import Adam

# Build the neural network
model = Sequential()
# Input layer (5 inputs)
model.add(Input(shape=(X_train.shape[1],)))

model.add(Dense(10, activation='relu')) 
# Additional hidden layer (optional)

model.add(Dense(5, activation='relu'))  # 5 neurons in the hidden layer

# Output layer (binary classification, so 1 neuron with sigmoid activation)
model.add(Dense(1, activation='sigmoid'))
print(model.summary())

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 10)                  │             180 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 5)                   │              55 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │               6 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 241 (964.00 B)

 Trainable params: 241 (964.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [4]:
#compiling the model(configuring the model)
model.compile(loss='binary_crossentropy',optimizer=Adam(learning_rate=0.01), metrics=['accuracy'])

In [5]:
#training the model
from keras.callbacks import EarlyStopping, ModelCheckpoint

# Create the callbacks
callback_a = ModelCheckpoint(filepath="my_best_model.keras", monitor='val_loss', save_best_only=True)
callback_b = EarlyStopping(monitor='val_loss', mode='min', patience=20, verbose=1)



In [6]:
history= model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=256,batch_size=20,callbacks=[callback_a,callback_b],verbose=1)

Epoch 1/256
19/19 ━━━━━━━━━━━━━━━━━━━━ 4s 39ms/step - accuracy: 0.5941 - loss: 15.0498 - val_accuracy: 0.7021 - val_loss: 1.5884
Epoch 2/256
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7620 - loss: 1.1145 - val_accuracy: 0.6915 - val_loss: 0.7378
Epoch 3/256
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8160 - loss: 0.4904 - val_accuracy: 0.9894 - val_loss: 0.1066
Epoch 4/256
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.9500 - loss: 0.2024 - val_accuracy: 0.9681 - val_loss: 0.0931
Epoch 5/256
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.9347 - loss: 0.1625 - val_accuracy: 0.9149 - val_loss: 0.1523
Epoch 6/256
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.9335 - loss: 0.1595 - val_accuracy: 0.9787 - val_loss: 0.0585
Epoch 7/256
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9642 - loss: 0.1021 - val_accuracy: 0.9787 - val_loss: 0.0561
Epoch 8/256
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9549 - loss: 0.0998 - val_accuracy: 0.

19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9759 - loss: 0.0588 - val_accuracy: 0.9787 - val_loss: 0.0645
Epoch 52/256
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9732 - loss: 0.0676 - val_accuracy: 0.9787 - val_loss: 0.0345
Epoch 53/256
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9667 - loss: 0.0555 - val_accuracy: 0.9255 - val_loss: 0.1519
Epoch 54/256
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9730 - loss: 0.0704 - val_accuracy: 0.9362 - val_loss: 0.1010
Epoch 55/256
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9635 - loss: 0.1138 - val_accuracy: 1.0000 - val_loss: 0.0224
Epoch 56/256
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9369 - loss: 0.2965 - val_accuracy: 0.9681 - val_loss: 0.0653
Epoch 57/256
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9872 - loss: 0.0411 - val_accuracy: 0.9362 - val_loss: 0.1378
Epoch 58/256
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.9677 - loss: 0.0929 - val_accuracy: 1.0000 - val

19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.9750 - loss: 0.0868 - val_accuracy: 1.0000 - val_loss: 0.0098
Epoch 102/256
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9975 - loss: 0.0323 - val_accuracy: 1.0000 - val_loss: 0.0106
Epoch 103/256
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.9910 - loss: 0.0432 - val_accuracy: 1.0000 - val_loss: 0.0088
Epoch 104/256
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9971 - loss: 0.0282 - val_accuracy: 1.0000 - val_loss: 0.0121
Epoch 105/256
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9947 - loss: 0.0246 - val_accuracy: 1.0000 - val_loss: 0.0108
Epoch 106/256
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9911 - loss: 0.0307 - val_accuracy: 1.0000 - val_loss: 0.0111
Epoch 107/256
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9989 - loss: 0.0192 - val_accuracy: 1.0000 - val_loss: 0.0189
Epoch 108/256
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.9953 - loss: 0.0220 - val_accuracy: 1.000

19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9985 - loss: 0.0135 - val_accuracy: 1.0000 - val_loss: 0.0187
Epoch 152/256
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9962 - loss: 0.0154 - val_accuracy: 1.0000 - val_loss: 0.0052
Epoch 153/256
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9991 - loss: 0.0138 - val_accuracy: 1.0000 - val_loss: 0.0270
Epoch 154/256
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9985 - loss: 0.0247 - val_accuracy: 1.0000 - val_loss: 0.0100
Epoch 155/256
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9958 - loss: 0.0163 - val_accuracy: 1.0000 - val_loss: 0.0059
Epoch 156/256
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9994 - loss: 0.0084 - val_accuracy: 1.0000 - val_loss: 0.0086
Epoch 157/256
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9943 - loss: 0.0180 - val_accuracy: 1.0000 - val_loss: 0.0127
Epoch 158/256
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9914 - loss: 0.0251 - val_accuracy: 1.0000 

19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9921 - loss: 0.0364 - val_accuracy: 1.0000 - val_loss: 0.0138
Epoch 202/256
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9731 - loss: 0.0666 - val_accuracy: 1.0000 - val_loss: 0.0125
Epoch 203/256
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9980 - loss: 0.0228 - val_accuracy: 1.0000 - val_loss: 0.0035
Epoch 204/256
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9907 - loss: 0.0483 - val_accuracy: 1.0000 - val_loss: 0.0083
Epoch 205/256
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9952 - loss: 0.0290 - val_accuracy: 1.0000 - val_loss: 0.0064
Epoch 206/256
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9983 - loss: 0.0199 - val_accuracy: 1.0000 - val_loss: 0.0131
Epoch 207/256
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9996 - loss: 0.0115 - val_accuracy: 1.0000 - val_loss: 0.0046
Epoch 208/256
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9933 - loss: 0.0191 - val_accuracy: 1.0000 

In [14]:
#checking the learning curves
print(history.params)


{'verbose': 1, 'epochs': 256, 'steps': 19}


In [15]:
#evaluate the model
scores=model.evaluate(X_train,y_train)
print(model.metrics_names)
print(scores)
print("\n%s: %.2f%%" % (model.metrics_names[1],scores[1]*100))


12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9947 - loss: 0.0185 
['loss', 'compile_metrics']
[0.011504490859806538, 0.9973118305206299]

compile_metrics: 99.73%


In [16]:
#evaluating on validating set
scores=model.evaluate(X_test,y_test)
print("\n%s: %.2f%%" % (model.metrics_names[1],scores[1]*100))


3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 1.0000 - loss: 0.0036

compile_metrics: 100.00%


In [17]:
print(X_test[0:5])
print(y_test[0:5])

     2  0.0  0.0.1  0.0.2  0.0.3  0.0.4  0.0.5       0.0.6       0.0.7  \
55   2  0.0    0.0    0.0    0.0    0.0    0.0    0.000000    0.000000   
63   2  0.0    0.0    0.0    0.0    0.0    0.0   96.591150   96.591150   
33   2  0.0    0.0    0.0    0.0    0.0    0.0    0.000000    0.000000   
461  2  0.0    0.0    0.0    0.0    0.0    0.0  139.732184  139.732184   
72   2  0.0    0.0    0.0    0.0    0.0    0.0   90.415267   90.415267   

          0.0.8       0.0.9  0.0.10  0.0.11  0.0.12  0.0.13  0.0.14  0.0.15  
55    79.878591    0.000000     0.0     0.0     0.0     0.0     0.0     0.0  
63    83.478251   83.577068     0.0     0.0     0.0     0.0     0.0     0.0  
33   176.724037  163.864628     0.0     0.0     0.0     0.0     0.0     0.0  
461  156.939403  158.882775     0.0     0.0     0.0     0.0     0.0     0.0  
72    82.002406   86.428903     0.0     0.0     0.0     0.0     0.0     0.0  
55     1.0
63     1.0
33     0.0
461    0.0
72     1.0
Name: 0.0.16, dtype: float64


In [11]:
prediction=model.predict(X_test)

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step


In [12]:
print(prediction[0:10])

[[9.7242850e-01]
 [9.9828631e-01]
 [3.9375612e-19]
 [1.6502441e-03]
 [9.9988782e-01]
 [4.0570080e-07]
 [0.0000000e+00]
 [1.0664665e-02]
 [9.9995571e-01]
 [7.3840944e-08]]


In [13]:
print(prediction[0:10].round())

[[1.]
 [1.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]]
